# **Echtzeitbetriebssysteme (EBS)**
#### ~*Gün Yanik*

## Rate Monotonic Scheduling (RMS)
  * Je höher die Rate (d.h. je kleiner die Periode) eines Prozesses, desto höher seine statische Priorität
  * Rate = Kehrwert der Periode
  * Laufbereiter Prozess mit höherer Priorität unterbricht stets Prozess mit niedrigerer Priorität
  
  Optimal: **Nein**, gültiger Schedule wird nicht immer gefunden (s. letzte Folie)
  **ABER**: bei Benutzung statischer Prioritäten gibt es keinen Scheduling-Algorithmus mit besserenPlanungsergebnissen.
  
## Earliest Deadline First (EDF) ~ very common
  * Prozess mit kürzester Deadline wird eingeplant
  * bei kooperativem Scheduling: Einplanung nach Abschluss des laufenden Prozesses
  * bei präemptivem Scheduling: Einplanung sobald neuer Prozess bereit ist
  * geeignet für periodische und aperiodische Pläne
  
  Optimal:
  **Ja**, für präemptives Scheduling: wenn ein Schedule existiert, der keine Deadline verletzt, wird er mit präemptivem EDF Scheduling immer gefunden.
  **Nein**, für kooperatives Scheduling
  
## Least LaxityFirst (LLF)
 * Ordnung nach zur Verfügung stehenden Spielräumen (laxity)
 * Laxity= Differenz zwischen deadline und (restlicher) Ausführungszeit
 * der Prozess mit der kleinsten Laxity(least laxity) wird zuerst ausgeführt
    
   Optimal:
    **Ja**, für präemptivesScheduling
    **Nein**, für kooperatives Scheduling


In [49]:
class Process:
    def __init__(self, name: str, duration: int, period: int):
        self.name = name           # name e.g 'A'
        self.duration = duration   # WCET or simply duration of process
        self.period = period       # period , also deadline
        self.lax = period-duration # init laxation
        self.curr = duration       # init duration left for current period
        self.occ = 1               # count of times finished for lableing
        self.waiting = 0
    

In [133]:
import copy #for copy.deepcopy(x)

'''
    Class of schedulers.
    callable with Schedule("name", list{dtype:Process})
    
    ex. 
    A = Process('A', 1, 5)
    B = Process('B', 3, 7)
    C = Process('C', 3, 9)
    procsa = [A, B, C]
    schedules = Schedule("a", procsa)
    print(20*'--'+'RMS'+20*'--')
    schedules.rms(24)
    print(20*'--'+'EDF'+20*'--')
    schedules.edf(24)
    print(20*'--'+'LLF'+20*'--')
    schedules.llf(24)
    
    @Author: Guen Yanik
'''
class Schedule:
    def __init__(self, name, proc):
        self.name = name
        self.processes = proc
    
    # print schedule properly
    def printschedule(self, liste):
        for i in range(len(liste)):
            print(str(i)+'|', end=' ')
        print(' ')
        for i,x in enumerate(liste):
            if i>10:
                print(' '+x, end=' ')
            else:
                print(x, end=' ')
                

    def rms(self, iterations): 
        procs = copy.deepcopy(self.processes)
        res = []
        for i in range(1,iterations+1):
                
            cur = min(procs, key=lambda x: x.period if x.curr>0 else 999999)
            cur.curr -= 1
            cur.waiting = 0
            res.append(cur.name+str(cur.occ))
        
            for c in procs:
                if ((i)%c.period)==0 and i!=0:
                    c.curr = c.duration
                    c.occ += 1
                    c.wating = 0
                if(cur.name != c.name):
                    c.waiting += 1
            procs.sort(key=lambda x: (x.occ, -x.waiting))
            procs.insert(0,procs.pop(procs.index(cur)))
        self.printschedule(res)
   #     return res
    
    # PREIMTIVE !!!!
    def edf(self, iterations):
        procs = copy.deepcopy(self.processes)
        res = []
        for i in range(1,iterations+1):
            
            cur = min(procs, key=lambda x: ((x.occ)*x.period) if x.curr>0 else 999999)
            cur.curr -= 1
            cur.waiting = 0
            res.append(cur.name+str(cur.occ))

            for c in procs:
                if ((i%c.period)==0 and i!=0) or (c.curr==0 and i==0) :
                    c.curr = c.duration
                    c.occ += 1
                    c.wating = 0
                if(cur.name != c.name):
                    c.waiting += 1
                    
            procs.sort(key=lambda x: (x.occ, -x.waiting))
            procs.insert(0,procs.pop(procs.index(cur)))
        self.printschedule(res)
       # return res

                    
    def llf(self, iterations, laxity_on=False):
        procs = copy.deepcopy(self.processes)
        res = []
        for i in range(1,iterations+1):
            for cc in procs:

                if i != 1:
                    if cc.curr!=0 and cc.name is not cur.name:
                        #print(i, cc.name, cur.name, cc.lax)
                        cc.lax -= 1
                if laxity_on:
                    if cc.curr==0:
                        print(i-1, cc.name, '-')
                    else:
                        print(i-1, cc.name, cc.lax)
                    
            cur = min(procs, key=lambda x: x.lax if x.curr>0 else 999999)
            cur.curr -= 1
            cur.waiting= 0
            res.append(cur.name+str(cur.occ))
            for c in procs:
                if ((i%c.period)==0 and i!=0) or (c.curr==0 and i==0) :
                    c.curr = c.duration
                    c.lax = c.period-c.duration+1
                    c.occ += 1
                    c.wating = 0
                if not(cur.name in c.name):
                    c.waiting += 1
           
            procs.sort(key=lambda x: (x.occ, -x.waiting))
            procs.insert(0,procs.pop(procs.index(cur)))
        self.printschedule(res)
    #    return res
    

In [134]:
# TUT AUFGABE 1
A = Process('A', 1, 5)
B = Process('B', 3, 7)
C = Process('C', 3, 9)
procsa = [A, B, C] 

schedules = Schedule("a", procsa)

print(20*'--'+'RMS'+20*'--')
schedules.rms(24)
print(20*'--'+'EDF'+20*'--')
schedules.edf(24)
print(20*'--'+'LLF'+20*'--')
schedules.llf(24)

----------------------------------------RMS----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|  
A1 B1 B1 B1 C1 A2 C1 B2 B2 B2 A3  C2  C2  C2  B3  A4  B3  B3  C3  C3  A5  B4  B4  B4 ----------------------------------------EDF----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|  
A1 B1 B1 B1 C1 C1 C1 A2 B2 B2 B2  A3  C2  C2  C2  A4  B3  B3  B3  C3  A5  C3  C3  B4 ----------------------------------------LLF----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|  
A1 B1 B1 B1 C1 C1 C1 A2 B2 B2 B2  A3  C2  C2  C2  B3  B3  A4  B3  C3  A5  C3  B4  B4 

In [125]:
#TUT AUFGABE 2

# MINIMALE ABWEICHUNG MÖGLICH BEI PATT 
A = Process('A', 3, 9)
B = Process('B', 5, 18)
C = Process('C', 4, 12)
procsa = [A, B, C]

schedules = Schedule("b", procsa)

print(20*'--'+'RMS'+20*'--')
schedules.rms(33)
print(20*'--'+'EDF'+20*'--')
schedules.edf(34)
print(20*'--'+'LLF'+20*'--')
schedules.llf(34) # TURN ON LAXIATION TABLE MODE

----------------------------------------RMS----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30| 31| 32|  
A1 A1 A1 C1 C1 C1 C1 B1 B1 A2 A2  A2  C2  C2  C2  C2  B1  B1  A3  A3  A3  B2  B2  B2  C3  C3  C3  A4  A4  A4  C3  B2  B2 ----------------------------------------EDF----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30| 31| 32| 33|  
A1 A1 A1 C1 C1 C1 C1 B1 B1 B1 B1  B1  A2  A2  A2  C2  C2  C2  C2  A3  A3  A3  B2  B2  B2  B2  B2  C3  C3  C3  C3  A4  A4  A4 ----------------------------------------LLF----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30| 31| 32| 33|  
A1 A1 A1 C1 C1 C1 C1 B1 B1 B1 A2  A2  B1  B1  A2  C2  C2  C2  C2  A3  A3  A3  B2  B2  C3  C3  B2  A4  A4  C3  B2  B2  A4  

In [126]:
# TUT AUFGABE 3
A = Process('A', 2, 6)
B = Process('B', 3, 10)
C = Process('C', 4, 13)
procsa = [A, B, C]

schedules = Schedule("c", procsa)

print(20*'--'+'RMS'+20*'--')
schedules.rms(25)
print(20*'--'+'EDF'+20*'--')
schedules.edf(25)
print(20*'--'+'LLF'+20*'--')
schedules.llf(25)

----------------------------------------RMS----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24|  
A1 A1 B1 B1 B1 C1 A2 A2 C1 C1 B2  B2  A3  A3  B2  C2  C2  C2  A4  A4  B3  B3  B3  C2  A5 ----------------------------------------EDF----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24|  
A1 A1 B1 B1 B1 C1 A2 A2 C1 C1 C1  B2  A3  A3  B2  B2  C2  C2  A4  A4  C2  C2  B3  B3  B3 ----------------------------------------LLF----------------------------------------
0| 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24|  
A1 A1 B1 B1 B1 C1 C1 A2 A2 C1 C1  B2  A3  A3  B2  B2  C2  C2  A4  A4  C2  C2  B3  B3  A5 